# Bayes Decision Theory

In [4]:
import numpy as np
import pandas as pd
from math import log
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
from collections import Counter

import data and data preprocessing "?" are replace by most occuring element in the column 

In [5]:
df = pd.read_csv("census-income.data")
dataToProcess = ['ACLSWKR','ADTIND','ADTOCC','AHGA','AHSCOL','AMARITL','AMJIND','AMJOCC','ARACE','AREORGN','ASEX','AUNMEM','AUNTYPE','AWKSTAT','FILESTAT','GRINREG','GRINST','HHdataMX','HHDREL','MIGMTR1','MIGMTR3','MIGMTR4','MIGSAME','MIGSUN','PARENT','PEFNTVTY','PEMNTVTY','PENATVTY','PRCITSHP','SEOTR','VETQVA','VETYN','YEAR', 'INCOME']
heaDer = ["AAGE","ACLSWKR","ADTIND","ADTOCC","AHGA","AHRSPAY","AHSCOL","AMARITL","AMJIND","AMJOCC","ARACE","AREORGN","ASEX","AUNMEM","AUNTYPE","AWKSTAT","CAPGAIN","CAPLOSS","DIVVAL","FILESTAT","GRINREG","GRINST","HHDFMX","HHDREL","MIGMTR1","MIGMTR3","MIGMTR4","MIGSAME","MIGSUN","NOEMP","PARENT","PEARNVAL","PEFNTVTY","PEMNTVTY","PENATVTY","PRCITSHP","SEOTR","VETQVA","VETYN","WKSWORK","YEAR","INCOME"]
for i in range(len(heaDer)):
	x = Counter(np.array(df[heaDer[i]]))
	if(x[" ?"] != 0):
		mostOccr = x.most_common()[0][0]
		if(mostOccr != " ?"):
			replaceValue = x.most_common()[0][0]
			replaceValue = replaceValue.strip()
			df[heaDer[i]] = df[heaDer[i]].str.replace("?",replaceValue)
		else:
			replaceValue = x.most_common()[1][0]
			replaceValue = replaceValue.strip()
			df[heaDer[i]] = df[heaDer[i]].str.replace("?",replaceValue)
df["MIGMTR1"] = str(df["MIGMTR1"])
df = df.loc[:,dataToProcess]
print(df)

                                ACLSWKR  ADTIND  ADTOCC  \
0                       Not in universe       0       0   
1        Self-employed-not incorporated       4      34   
2                       Not in universe       0       0   
3                       Not in universe       0       0   
4                       Not in universe       0       0   
5                               Private      40      10   
6                               Private      34       3   
7                               Private       4      40   
8                      Local government      43      26   
9                               Private       4      37   
10                      Not in universe       0       0   
11                      Not in universe       0       0   
12                              Private       4      34   
13                              Private      37      31   
14                              Private      24      12   
15                      Not in universe       0       0 

training and testing of data

In [ ]:
Accuracy = np.array([],dtype=float)
for i in range(30):
	kf = KFold(n_splits=10,shuffle=True)
	df = np.array(df)
	accuracy = 0.0
	for train, test in kf.split(df):
		trainData = pd.DataFrame(df[train], columns=dataToProcess)
		testData = pd.DataFrame(df[test], columns=dataToProcess)
        
		less50K = trainData.loc[trainData.INCOME == " - 50000.", :]
		more50K = trainData.loc[trainData.INCOME == " 50000+.", :]
		
        numLess = len(less50K)
		numMore = len(more50K)
		
        probLess = log(numLess/len(trainData))  #a-priori probability (log probabilities)
		probMore = log(numMore/len(trainData))
        
		less50KDict = {}
		more50KDict = {}
		
        for columnName in dataToProcess:
			less50KDict[columnName] = dict(less50K[columnName].value_counts()/numLess)
			more50KDict[columnName] = dict(more50K[columnName].value_counts()/numMore)
		#testing data
		
        tempAcc = 0.0
		
        for i in range(len(testData)):
			X = testData.iloc[i:i+1]
			posteriorLess = probLess
			posteriorMore = probMore
			for columnName in dataToProcess[:-1]:
				if X[columnName].iloc[0] in less50KDict[columnName].keys():
					posteriorLess += log(less50KDict[columnName][X[columnName].iloc[0]])
				if X[columnName].iloc[0] in more50KDict[columnName].keys():
					posteriorMore += log(more50KDict[columnName][X[columnName].iloc[0]])
                    
			if posteriorLess > posteriorMore and X['INCOME'].iloc[0] == " - 50000.":
				tempAcc += 1
			elif posteriorMore > posteriorLess and X['INCOME'].iloc[0] == " 50000+.": 
				tempAcc += 1
		tempAcc /= len(testData)
		accuracy += tempAcc
	accuracy /= 10
	print(accuracy)
	Accuracy = np.append(Accuracy, accuracy)
print("Mean ", np.mean(Accuracy))
print("StandardDevation: ", np.std(Accuracy))

.77518377862
.76559370651
.77438365963
.75218377340
.78317371062
.76513470867
.
.
.
.
.
.
.


Mean  0.76923978662
StandardDevation: .000110539654